<a href="https://colab.research.google.com/github/jessicaMorale/RI-busquedaB-MoralesJ/blob/main/Notebook/01busquedaBasica_RI_Morales_Jessica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Escuela Politécnica Nacional**
- Nombre: Morales Jessica
- Grupo: GR1CC

# Ejercicio 1: Introducción a Recuperación de Información

## Objetivo de la práctica
- Entender el problema de **buscar información** en colecciones de texto.
- Comprender por qué se necesita un **índice invertido** en recuperación de información.
- Programar una primera solución manual y luego optimizarla con un índice.
- Evaluar la mejora en tiempos de búsqueda cuando usamos estructuras adecuadas.

## Parte 1: Búsqueda lineal en documentos

### Actividad
1. Se te proporcionará un conjunto de documentos de texto.
2. Escribe una función que:
   - Lea todos los documentos.
   - Busque una palabra ingresada por el usuario.
   - Muestre en qué documentos aparece la palabra.

In [12]:
import requests

def descargar_archivos(urls):
    documentos = {}
    for url in urls:
        archivo_nombre = url.split('/')[-1]
        try:
            respuesta = requests.get(url)
            respuesta.raise_for_status()
            documentos[archivo_nombre] = respuesta.text
        except requests.exceptions.RequestException as e:
            print(f"Error al descargar {archivo_nombre} desde {url}: {e}")
    return documentos

def buscar_palabra_con_lineas(documentos, palabra):
    resultados = {}
    for archivo, contenido in documentos.items():
        resultados_archivo = []
        for numero_linea, linea in enumerate(contenido.splitlines(), 1):
            if palabra.lower() in linea.lower():
                resultados_archivo.append(numero_linea)
        if resultados_archivo:
            resultados[archivo] = resultados_archivo
    return resultados

# Enlaces del repositorio
urls = [
    "https://github.com/ivan-carrera/ir25a/blob/main/data/01_corpus_turismo.txt",
    "https://github.com/ivan-carrera/ir25a/blob/main/data/01_corpus_turismo_500.txt"
]
documentos = descargar_archivos(urls)
palabra = input("Ingrese una palabra para buscar: ")
resultados_con_lineas = buscar_palabra_con_lineas(documentos, palabra)

if resultados_con_lineas:
    print(f"La palabra '{palabra}' aparece en los siguientes documentos y líneas:")
    for archivo, lineas in resultados_con_lineas.items():
        print(f"- {archivo}: Líneas {lineas}")
else:
    print(f"La palabra '{palabra}' no se encontró en ningún documento.")

Ingrese una palabra para buscar: Ecuador
La palabra 'Ecuador' aparece en los siguientes documentos y líneas:
- 01_corpus_turismo.txt: Líneas [1722, 1828]
- 01_corpus_turismo_500.txt: Líneas [1722, 1828]


## Parte 2: Construcción de un índice invertido

### Actividad
1. Escribe un programa que:
   - Recorra todos los documentos.
   - Construya un **índice invertido**, es decir, un diccionario donde:
     - Cada palabra clave apunta a una lista de documentos donde aparece.

2. Escribe una nueva función de búsqueda que:
   - Consulte directamente el índice para encontrar los documentos relevantes.
   - Sea mucho más rápida que la búsqueda lineal.

In [ ]:
import requests
import os

def descargar_archivos(urls):
    documentos = {}
    for url in urls:
        archivo_nombre = url.split('/')[-1]
        respuesta = requests.get(url, stream=True)
        if respuesta.status_code == 200:

            contenido = respuesta.content.decode('utf-8')
            lineas = contenido.splitlines()
            documentos[archivo_nombre] = lineas
        else:
            print(f"Error al descargar {archivo_nombre} desde {url}")
    return documentos

def busqueda_lineal(documentos, palabra):
    resultados = []
    for archivo, contenido in documentos.items():
        contenido_str = '\n'.join(contenido)
        if palabra.lower() in contenido_str.lower():
            resultados.append(archivo)
    return resultados


def construir_indice_invertido(documentos):
    indice_invertido = {}
    for nombre_archivo, contenido in documentos.items():
        for numero_linea, linea in enumerate(contenido, start=1):
            palabras = linea.split()
            for palabra in palabras:
                palabra = palabra.lower()
                if palabra not in indice_invertido:
                    indice_invertido[palabra] = []
                indice_invertido[palabra].append((nombre_archivo, numero_linea))
    return indice_invertido


def buscar_en_indice(indice_invertido, palabra):
    palabra = palabra.lower()
    return indice_invertido.get(palabra, [])



urls = [
    "https://raw.githubusercontent.com/ivan-carrera/ir25a/main/data/01_corpus_turismo.txt",
    "https://raw.githubusercontent.com/ivan-carrera/ir25a/main/data/01_corpus_turismo_500.txt"
]

documentos = descargar_archivos(urls)

# Busqueda lineal
palabra_busqueda = input("Ingrese una palabra para buscar: ")
resultado_lineal = busqueda_lineal(documentos, palabra_busqueda)
print(f"La palabra '{palabra_busqueda}' aparece en los documentos: {resultado_lineal}")

# Índice invertido
indice_invertido = construir_indice_invertido(documentos)
palabra_busqueda_indice = input("Ingresa la palabra a buscar en el índice: ")
resultados_indice = buscar_en_indice(indice_invertido, palabra_busqueda_indice)

if resultados_indice:
    print("La palabra '{}' se encontró en los siguientes documentos:".format(palabra_busqueda_indice))
    for documento, linea in resultados_indice:
        print("Documento: {}, Línea: {}".format(documento, linea))
else:
    print("No se encontró la palabra '{}' en ningún documento.".format(palabra_busqueda_indice))


Ingrese una palabra para buscar: perfecto
La palabra 'perfecto' aparece en los documentos: ['01_corpus_turismo_500.txt']
Ingresa la palabra a buscar en el índice: Ecuador
La palabra 'Ecuador' se encontró en los siguientes documentos:
Documento: 01_corpus_turismo.txt, Línea: 1
Documento: 01_corpus_turismo_500.txt, Línea: 5
Documento: 01_corpus_turismo_500.txt, Línea: 22
Documento: 01_corpus_turismo_500.txt, Línea: 30
Documento: 01_corpus_turismo_500.txt, Línea: 37
Documento: 01_corpus_turismo_500.txt, Línea: 41
Documento: 01_corpus_turismo_500.txt, Línea: 45
Documento: 01_corpus_turismo_500.txt, Línea: 76
Documento: 01_corpus_turismo_500.txt, Línea: 109
Documento: 01_corpus_turismo_500.txt, Línea: 110
Documento: 01_corpus_turismo_500.txt, Línea: 122
Documento: 01_corpus_turismo_500.txt, Línea: 127
Documento: 01_corpus_turismo_500.txt, Línea: 130
Documento: 01_corpus_turismo_500.txt, Línea: 156
Documento: 01_corpus_turismo_500.txt, Línea: 163
Documento: 01_corpus_turismo_500.txt, Línea: 

## Parte 3: Evaluación de tiempos de búsqueda
### Actividad

1. Realiza la búsqueda de varias palabras usando:
      -  Corpus pequeño: 16 documentos (turismo en Ecuador).
      -  Corpus grande: 500 documentos (versión ampliada).
2. Mide el tiempo de ejecución:
      -  Para búsqueda lineal.
      -  Para búsqueda usando índice invertido.
      -  Grafica o presenta los resultados en una tabla comparativa.

### Ejemplo de palabras para buscar
- quito
- montañita
- feriado
- playas
- aventura
- galápagos

In [3]:
import time
import requests
import os
from prettytable import PrettyTable
def descargar_archivos(urls):
    documentos = {}
    for url in urls:
        archivo_nombre = url.split('/')[-1]
        respuesta = requests.get(url, stream=True)
        if respuesta.status_code == 200:
            contenido = respuesta.content.decode('utf-8')
            lineas = contenido.splitlines()
            documentos[archivo_nombre] = lineas
        else:
            print(f"Error al descargar {archivo_nombre} desde {url}")
    return documentos

def busqueda_lineal(documento, palabra):
    contenido_str = '\n'.join(documento)
    return palabra.lower() in contenido_str.lower()

def construir_indice_invertido(documentos):
    indice_invertido = {}
    for nombre_archivo, contenido in documentos.items():
        for numero_linea, linea in enumerate(contenido, start=1):
            palabras = linea.split()
            for palabra in palabras:
                palabra = palabra.lower()
                if palabra not in indice_invertido:
                    indice_invertido[palabra] = []
                indice_invertido[palabra].append((nombre_archivo, numero_linea))
    return indice_invertido

def buscar_en_indice(indice_invertido, palabra, documento_nombre):
    palabra = palabra.lower()
    resultados = indice_invertido.get(palabra, [])
    return any(doc == documento_nombre for doc, _ in resultados)


urls = [
    "https://raw.githubusercontent.com/ivan-carrera/ir25a/main/data/01_corpus_turismo.txt",
    "https://raw.githubusercontent.com/ivan-carrera/ir25a/main/data/01_corpus_turismo_500.txt"
]

documentos = descargar_archivos(urls)
indice_invertido = construir_indice_invertido(documentos)

palabras_busqueda = input("Ingrese palabras para buscar separadas por espacios: ").split()

# Creación de la tabla
table = PrettyTable()
table.field_names = ["Palabra", "Documento", "Método de Búsqueda", "Tiempo (s)", "Resultado"]


for palabra_busqueda in palabras_busqueda:

    for nombre_archivo, contenido in documentos.items():
        # Búsqueda lineal
        inicio_lineal = time.time()
        resultado_lineal = busqueda_lineal(contenido, palabra_busqueda)
        fin_lineal = time.time()
        tiempo_lineal = fin_lineal - inicio_lineal

        # Búsqueda con índice invertido
        inicio_indice = time.time()
        resultado_indice = buscar_en_indice(indice_invertido, palabra_busqueda, nombre_archivo)
        fin_indice = time.time()
        tiempo_indice = fin_indice - inicio_indice


        table.add_row([palabra_busqueda, nombre_archivo, "Lineal", f"{tiempo_lineal:.6f}", "Encontrado" if resultado_lineal else "No encontrado"])
        table.add_row([palabra_busqueda, nombre_archivo, "Índice Invertido", f"{tiempo_indice:.6f}", "Encontrado" if resultado_indice else "No encontrado"])

print(table)


Ingrese palabras para buscar separadas por espacios: quito montañita feriado playa aventuras galápagos
+-----------+---------------------------+--------------------+------------+---------------+
|  Palabra  |         Documento         | Método de Búsqueda | Tiempo (s) |   Resultado   |
+-----------+---------------------------+--------------------+------------+---------------+
|   quito   |   01_corpus_turismo.txt   |       Lineal       |  0.000023  |   Encontrado  |
|   quito   |   01_corpus_turismo.txt   |  Índice Invertido  |  0.000007  |   Encontrado  |
|   quito   | 01_corpus_turismo_500.txt |       Lineal       |  0.000430  |   Encontrado  |
|   quito   | 01_corpus_turismo_500.txt |  Índice Invertido  |  0.000004  |   Encontrado  |
| montañita |   01_corpus_turismo.txt   |       Lineal       |  0.000017  |   Encontrado  |
| montañita |   01_corpus_turismo.txt   |  Índice Invertido  |  0.000003  |   Encontrado  |
| montañita | 01_corpus_turismo_500.txt |       Lineal       |  0.000

## Parte 4:
### Actividad
1. Modifica el índice para que ignore mayúsculas/minúsculas (por ejemplo, "Playa" y "playa" deben considerarse iguales).
2. Permite consultas de múltiples términos (ejemplo: buscar documentos que contengan "playa" y "turismo").
3. Calcula el _speedup_

In [14]:
import time
import requests
from collections import namedtuple

# Definimos una estructura para almacenar los resultados de cada búsqueda
ResultadoBusqueda = namedtuple("ResultadoBusqueda", ["palabra", "tiempo_lineal", "tiempo_indice"])

# URLs de los documentos (asumiendo que están en línea)
urls = [
    "https://raw.githubusercontent.com/ivan-carrera/ir25a/main/data/01_corpus_turismo_500.txt"
]


def descargar_archivos(urls):
    documentos = {}
    for url in urls:
        archivo_nombre = url.split('/')[-1]
        try:
            respuesta = requests.get(url)
            respuesta.raise_for_status()
            documentos[archivo_nombre] = respuesta.text.splitlines()
        except requests.exceptions.RequestException as e:
            print(f"Error al descargar {archivo_nombre} desde {url}: {e}")
    return documentos

#Función de busqueda de linea
def busqueda_lineal(documento, palabra):
    return palabra in '\n'.join(documento)

# Función para construir el índice invertido
def construir_indice_invertido(documentos):
    indice_invertido = {}
    for nombre_archivo, contenido in documentos.items():
        for numero_linea, linea in enumerate(contenido, start=1):
            palabras = linea.split()
            for palabra in palabras:
                palabra_lower = palabra.lower()
                if palabra_lower not in indice_invertido:
                    indice_invertido[palabra_lower] = []
                indice_invertido[palabra_lower].append((nombre_archivo, numero_linea))
    return indice_invertido

# Función para buscar en el índice invertido (adaptada)
def buscar_en_indice(indice_invertido, palabra):
    palabra_lower = palabra.lower()
    return indice_invertido.get(palabra_lower, [])

# Función para comparar las búsquedas y almacenar los resultados
def comparar_busquedas(documentos, indice_invertido, palabras_busqueda):
    resultados_comparacion = []
    for palabra in palabras_busqueda:
        # Búsqueda lineal
        inicio_lineal = time.time()
        for nombre_archivo, contenido in documentos.items():
            busqueda_lineal(contenido, palabra)
        tiempo_lineal = time.time() - inicio_lineal

        # Búsqueda con índice invertido
        inicio_indice = time.time()
        buscar_en_indice(indice_invertido, palabra)
        tiempo_indice = time.time() - inicio_indice

        resultados_comparacion.append(ResultadoBusqueda(palabra, tiempo_lineal, tiempo_indice))
    return resultados_comparacion

# Ejecución principal
documentos = descargar_archivos(urls)
indice_invertido = construir_indice_invertido(documentos)

# Obtener palabras de búsqueda del usuario
palabras_busqueda_input = input("Ingrese palabras para buscar separadas por espacios: ").split()

resultados = comparar_busquedas(documentos, indice_invertido, palabras_busqueda_input)

# Imprimir los resultados
for resultado in resultados:
    print(f"Palabra: {resultado.palabra}, Tiempo lineal: {resultado.tiempo_lineal:.6f}, Tiempo índice: {resultado.tiempo_indice:.6f}")

Ingrese palabras para buscar separadas por espacios: quito montañita playa aventura galápagos
Palabra: quito, Tiempo lineal: 0.000069, Tiempo índice: 0.000003
Palabra: montañita, Tiempo lineal: 0.000053, Tiempo índice: 0.000003
Palabra: playa, Tiempo lineal: 0.000014, Tiempo índice: 0.000001
Palabra: aventura, Tiempo lineal: 0.000018, Tiempo índice: 0.000002
Palabra: galápagos, Tiempo lineal: 0.000044, Tiempo índice: 0.000001
